# Cluster in Residuen - Das verräterische Muster

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_12/cluster_in_residuen_demo.ipynb)

**Lernziel:** Verstehen, wie fehlende kategoriale Variablen zu Clustern in Residuenplots führen und warum das ein starkes Indiz für Modellverbesserungen ist.

**Aus VL12:** Cluster-Muster in Residuenplots zeigen getrennte Gruppen von Datenpunkten und sind ein starkes Indiz für eine **fehlende kategoriale Variable** im Modell.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

# Stil für bessere Plots
plt.style.use('default')
np.random.seed(42)

print("✓ Bibliotheken erfolgreich importiert")

## Szenario: Gehälter basierend auf Berufserfahrung

**Das Problem:** Wir modellieren Gehälter nur mit "Jahren Berufserfahrung", ignorieren aber "Ausbildungsniveau".

**Was passiert:** Es entstehen zwei getrennte Gruppen (Cluster) in den Residuen.

In [ ]:
# Simulierte Gehaltsdaten mit versteckter kategorialer Variable
n_samples = 200

# Berufserfahrung (Jahre)
years_experience = np.random.uniform(0, 20, n_samples)

# Kategoriale Variable: Ausbildung (50% ohne Studium, 50% mit Studium)
education = np.random.choice(['Ohne Studium', 'Mit Studium'], n_samples, p=[0.5, 0.5])

# ZWEI GETRENNTE LINIEN: Verschiedene Grundgehälter je nach Ausbildung
salary = np.where(
    education == 'Ohne Studium',
    35000 + 1500 * years_experience + np.random.normal(0, 3000, n_samples),  # Niedrigeres Grundgehalt
    55000 + 2500 * years_experience + np.random.normal(0, 4000, n_samples)   # Höheres Grundgehalt
)

# DataFrame erstellen
df = pd.DataFrame({
    'years_experience': years_experience,
    'education': education,
    'salary': salary
})

print(f"Datensatz erstellt: {len(df)} Personen")
print(f"Ausbildungsverteilung:")
print(df['education'].value_counts())
print(f"\nDurchschnittsgehälter:")
print(df.groupby('education')['salary'].mean().round(0))

## In den Daten "stecken zwei Linien"

Schauen wir uns die Daten zuerst an, **bevor** wir das problematische Modell bauen:

In [ ]:
# Visualisierung: Die "zwei getrennten Linien" sichtbar machen
plt.figure(figsize=(12, 8))

# Daten nach Ausbildung getrennt plotten
for education_level in df['education'].unique():
    data = df[df['education'] == education_level]
    color = 'blue' if education_level == 'Ohne Studium' else 'red'
    
    plt.scatter(data['years_experience'], data['salary'], 
               alpha=0.6, color=color, s=50, label=education_level)
    
    # Trendlinie für jede Gruppe
    z = np.polyfit(data['years_experience'], data['salary'], 1)
    p = np.poly1d(z)
    x_trend = np.linspace(data['years_experience'].min(), data['years_experience'].max(), 100)
    plt.plot(x_trend, p(x_trend), color=color, linewidth=3, alpha=0.8)

plt.xlabel('Jahre Berufserfahrung')
plt.ylabel('Gehalt (€)')
plt.title('ZWEI GETRENNTE LINIEN: Gehalt vs. Berufserfahrung\n(nach Ausbildung getrennt dargestellt)')
plt.legend()
plt.grid(True, alpha=0.3)

# Annotationen
plt.annotate('Höhere Linie:\nMit Studium', xy=(15, 95000), xytext=(12, 105000),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=12, color='red', weight='bold')

plt.annotate('Niedrigere Linie:\nOhne Studium', xy=(15, 65000), xytext=(5, 50000),
            arrowprops=dict(arrowstyle='->', color='blue'),
            fontsize=12, color='blue', weight='bold')

plt.tight_layout()
plt.show()

print("📊 ERKLÄRUNG DER 'ZWEI GETRENNTEN LINIEN':")
print("• Blaue Linie: Ohne Studium - niedrigeres Grundgehalt, langsamerer Anstieg")
print("• Rote Linie: Mit Studium - höheres Grundgehalt, steilerer Anstieg")
print("• Beide Gruppen folgen linearen Trends, aber mit verschiedenen Parametern")
print("• Ein einfaches Modell 'sieht' nur eine Mischung aus beiden Gruppen")

## Problematisches Modell: Ohne kategoriale Variable

Jetzt modellieren wir **nur** mit Berufserfahrung und ignorieren die Ausbildung:

In [ ]:
# SCHLECHTES MODELL: Nur Berufserfahrung, Ausbildung ignoriert
X_bad = df[['years_experience']]  # Nur eine Variable!
y = df['salary']

# Lineares Modell trainieren
model_bad = LinearRegression()
model_bad.fit(X_bad, y)

# Vorhersagen und Residuen berechnen
y_pred_bad = model_bad.predict(X_bad)
residuals_bad = y - y_pred_bad

print("🚨 SCHLECHTES MODELL (ohne Ausbildung):")
print(f"R² Score: {model_bad.score(X_bad, y):.3f}")
print(f"Mittlerer absoluter Fehler: {np.mean(np.abs(residuals_bad)):.0f} €")
print(f"Modell: Gehalt = {model_bad.intercept_:.0f} + {model_bad.coef_[0]:.0f} × Jahre")

In [ ]:
# Visualisierung des schlechten Modells
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Links: Modell-Fit
ax1.scatter(df['years_experience'], df['salary'], alpha=0.6, color='gray', s=30)
x_line = np.linspace(0, 20, 100)
y_line = model_bad.intercept_ + model_bad.coef_[0] * x_line
ax1.plot(x_line, y_line, color='green', linewidth=3, label='Einfache Regressionslinie')

ax1.set_xlabel('Jahre Berufserfahrung')
ax1.set_ylabel('Gehalt (€)')
ax1.set_title('Schlechtes Modell: Eine Linie für alle\n(Ignoriert Ausbildungsunterschiede)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Rechts: RESIDUENPLOT - Hier sehen wir die Cluster!
colors = ['blue' if edu == 'Ohne Studium' else 'red' for edu in df['education']]
ax2.scatter(y_pred_bad, residuals_bad, c=colors, alpha=0.7, s=40)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.8)
ax2.set_xlabel('Vorhersage (€)')
ax2.set_ylabel('Residuen (€)')
ax2.set_title('🚨 CLUSTER IN RESIDUEN!\n(Verräterisches Muster)')
ax2.grid(True, alpha=0.3)

# Legende für Residuenplot
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='blue', label='Ohne Studium'),
                   Patch(facecolor='red', label='Mit Studium')]
ax2.legend(handles=legend_elements)

# Cluster markieren
ax2.annotate('Cluster 1:\nNegative Residuen\n(Ohne Studium)', 
            xy=(60000, -15000), xytext=(45000, -25000),
            arrowprops=dict(arrowstyle='->', color='blue'),
            fontsize=10, color='blue', weight='bold')

ax2.annotate('Cluster 2:\nPositive Residuen\n(Mit Studium)', 
            xy=(80000, 20000), xytext=(95000, 30000),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=10, color='red', weight='bold')

plt.tight_layout()
plt.show()

print("🔍 WAS SEHEN WIR IM RESIDUENPLOT?")
print("• CLUSTER 1 (blau): Systematisch negative Residuen → Modell überschätzt")
print("• CLUSTER 2 (rot): Systematisch positive Residuen → Modell unterschätzt")
print("• Die Cluster entsprechen den Ausbildungsgruppen!")
print("• Das ist das 'verräterische Muster' - es verrät die fehlende Variable")

## Lösung: Dummy-Variablen hinzufügen

Jetzt fügen wir die fehlende kategoriale Variable als **Dummy-Variable** hinzu:

In [ ]:
# GUTES MODELL: Mit Dummy-Variable für Ausbildung
df_with_dummy = df.copy()

# Dummy-Variable erstellen (One-Hot-Encoding)
df_with_dummy['ist_studium'] = (df['education'] == 'Mit Studium').astype(int)

print("📊 DUMMY-VARIABLE ERKLÄRT:")
print("Original kategoriale Variable:")
print(df[['education']].head())
print("\nAls Dummy-Variable:")
print(df_with_dummy[['education', 'ist_studium']].head())
print("\n• ist_studium = 1 → Mit Studium")
print("• ist_studium = 0 → Ohne Studium (Referenzkategorie)")

# Neues Modell mit beiden Variablen
X_good = df_with_dummy[['years_experience', 'ist_studium']]
model_good = LinearRegression()
model_good.fit(X_good, y)

# Vorhersagen und Residuen
y_pred_good = model_good.predict(X_good)
residuals_good = y - y_pred_good

print(f"\n✅ GUTES MODELL (mit Ausbildung):")
print(f"R² Score: {model_good.score(X_good, y):.3f}")
print(f"Mittlerer absoluter Fehler: {np.mean(np.abs(residuals_good)):.0f} €")
print(f"\nModell-Gleichung:")
print(f"Gehalt = {model_good.intercept_:.0f} + {model_good.coef_[0]:.0f} × Jahre + {model_good.coef_[1]:.0f} × ist_studium")
print(f"\nInterpretation:")
print(f"• Grundgehalt ohne Studium: {model_good.intercept_:.0f} €")
print(f"• Gehaltssteigerung pro Jahr: {model_good.coef_[0]:.0f} €")
print(f"• Studiums-Bonus: {model_good.coef_[1]:.0f} € extra")

In [ ]:
# Vergleich: Residuenplots vorher vs. nachher
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Links: Schlechtes Modell (mit Clustern)
colors = ['blue' if edu == 'Ohne Studium' else 'red' for edu in df['education']]
ax1.scatter(y_pred_bad, residuals_bad, c=colors, alpha=0.7, s=40)
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.8)
ax1.set_xlabel('Vorhersage (€)')
ax1.set_ylabel('Residuen (€)')
ax1.set_title('🚨 VORHER: Cluster-Muster\n(Fehlende Variable)')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(-30000, 30000)

# Cluster hervorheben
#from matplotlib.patches import Ellipse
#ellipse1 = Ellipse((65000, -12000), 20000, 18000, alpha=0.2, color='blue')
#ellipse2 = Ellipse((75000, 15000), 25000, 20000, alpha=0.2, color='red')
#ax1.add_patch(ellipse1)
#ax1.add_patch(ellipse2)
ax1.text(65000, -12000, 'Cluster 1', ha='center', va='center', weight='bold', color='blue')
ax1.text(75000, 15000, 'Cluster 2', ha='center', va='center', weight='bold', color='red')

# Rechts: Gutes Modell (ohne Cluster)
ax2.scatter(y_pred_good, residuals_good, c=colors, alpha=0.7, s=40)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.8)
ax2.set_xlabel('Vorhersage (€)')
ax2.set_ylabel('Residuen (€)')
ax2.set_title('✅ NACHHER: Zufällige Streuung\n(Mit Dummy-Variable)')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(-30000, 30000)

# Legende
legend_elements = [Patch(facecolor='blue', label='Ohne Studium'),
                   Patch(facecolor='red', label='Mit Studium')]
ax1.legend(handles=legend_elements, loc='upper left')
ax2.legend(handles=legend_elements, loc='upper left')

plt.tight_layout()
plt.show()

print("🎯 VERBESSERUNG DURCH DUMMY-VARIABLEN:")
print(f"• R² vorher: {model_bad.score(X_bad, y):.3f} → nachher: {model_good.score(X_good, y):.3f}")
print(f"• Fehler vorher: {np.mean(np.abs(residuals_bad)):.0f}€ → nachher: {np.mean(np.abs(residuals_good)):.0f}€")
print(f"• Cluster verschwunden: Residuen jetzt zufällig um 0 verteilt")

## Warum funktionieren Dummy-Variablen?

Das Modell lernt jetzt **separate Grundniveaus** für jede Kategorie:

In [ ]:
# Visualisierung: Wie das gute Modell funktioniert
plt.figure(figsize=(12, 8))

# Daten nach Ausbildung plotten
for education_level in df['education'].unique():
    data = df[df['education'] == education_level]
    color = 'blue' if education_level == 'Ohne Studium' else 'red'
    
    plt.scatter(data['years_experience'], data['salary'], 
               alpha=0.6, color=color, s=50, label=f'{education_level} (Daten)')

# Modell-Vorhersagen für beide Gruppen
x_range = np.linspace(0, 20, 100)

# Für "Ohne Studium" (ist_studium = 0)
X_ohne = np.column_stack([x_range, np.zeros(len(x_range))])
y_ohne = model_good.predict(X_ohne)
plt.plot(x_range, y_ohne, color='blue', linewidth=3, linestyle='--', 
         label='Modell: Ohne Studium')

# Für "Mit Studium" (ist_studium = 1)
X_mit = np.column_stack([x_range, np.ones(len(x_range))])
y_mit = model_good.predict(X_mit)
plt.plot(x_range, y_mit, color='red', linewidth=3, linestyle='--', 
         label='Modell: Mit Studium')

plt.xlabel('Jahre Berufserfahrung')
plt.ylabel('Gehalt (€)')
plt.title('✅ Gutes Modell: Separate Linien durch Dummy-Variablen\nModell lernt verschiedene Grundniveaus')
plt.legend()
plt.grid(True, alpha=0.3)

# Annotationen der Unterschiede
#plt.annotate(f'Studiums-Bonus:\n{model_good.coef_[1]:.0f} € extra', 
#            xy=(10, 80000), xytext=(15, 90000),
#            arrowprops=dict(arrowstyle='<->', color='green', lw=2),
#            fontsize=12, color='green', weight='bold',
#            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

plt.tight_layout()
plt.show()

print("🧠 WIE DUMMY-VARIABLEN FUNKTIONIEREN:")
print("\nDas Modell berechnet:")
print(f"• Ohne Studium: Gehalt = {model_good.intercept_:.0f} + {model_good.coef_[0]:.0f} × Jahre + {model_good.coef_[1]:.0f} × 0")
print(f"                      = {model_good.intercept_:.0f} + {model_good.coef_[0]:.0f} × Jahre")
print(f"\n• Mit Studium:  Gehalt = {model_good.intercept_:.0f} + {model_good.coef_[0]:.0f} × Jahre + {model_good.coef_[1]:.0f} × 1")
print(f"                      = {model_good.intercept_ + model_good.coef_[1]:.0f} + {model_good.coef_[0]:.0f} × Jahre")
print(f"\n➜ Zwei parallele Linien mit {model_good.coef_[1]:.0f}€ Abstand")
print(f"➜ Systematische Gruppenunterschiede werden erklärt")
print(f"➜ Residuen werden zufällig → Cluster verschwinden")

## Zusammenfassung: Cluster als Diagnosewerkzeug

**Was wir gelernt haben:**

In [ ]:
print("🎯 CLUSTER IN RESIDUEN - ZUSAMMENFASSUNG:")
print("="*50)

print("\n🔍 WAS SIND CLUSTER IN RESIDUEN?")
print("• Getrennte Gruppen von Datenpunkten im Residuenplot")
print("• Ein Cluster systematisch über 0, der andere systematisch unter 0")
print("• Zeigen, dass das Modell verschiedene Gruppen unterschiedlich behandelt")

print("\n❓ WAS BEDEUTEN 'ZWEI GETRENNTE LINIEN'?")
print("• Verschiedene Gruppen folgen verschiedenen linearen Beziehungen")
print("• Unterschiedliche Grundniveaus (Achsenabschnitte)")
print("• Eventuell auch unterschiedliche Steigungen")
print("• Ein einfaches Modell 'sieht' nur den Durchschnitt aus beiden")

print("\n🚨 WANN ENTSTEHEN CLUSTER?")
print("• Fehlende kategoriale Variable im Modell")
print("• Das Modell kann systematische Gruppenunterschiede nicht erklären")
print("• Residuen zeigen die 'übrig gebliebene' Gruppenstruktur")

print("\n✅ WIE LÖST MAN DAS PROBLEM?")
print("• Dummy-Variablen für kategoriale Features hinzufügen")
print("• One-Hot-Encoding: Text → binäre Zahlen (0/1)")
print("• Modell lernt separate Grundniveaus für jede Kategorie")
print("• Cluster verschwinden → zufällige Residuen um 0")

print("\n🎓 PRAKTISCHE ANWENDUNG:")
print("• Residuenplot nach kategorialen Variablen färben")
print("• Cluster = Hinweis auf vergessene kategoriale Variable")
print("• Systematische Verbesserung statt zufälliges Probieren")
print("• Modell-Diagnostik ist wichtiger als Algorithmus-Optimierung!")

print("\n💡 MERKSATZ:")
print('"Cluster in Residuen verraten fehlende kategoriale Variablen"')